In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import os
from PIL import Image
from shutil import copyfile

# Data Reading

In [2]:
COMMON_PART = '/home/bohdan/histologie/data/doctor_annotations/'
PATH_TO_FLOODFILLED_MASKS = os.path.join(COMMON_PART, 'floodfilled_masks')
PATH_TO_BINARY_MASKS = os.path.join(COMMON_PART, 'visible_binary_masks')

In [3]:
floodfilled_mask_names = os.listdir(PATH_TO_FLOODFILLED_MASKS)
binary_mask_names = os.listdir(PATH_TO_BINARY_MASKS)


In [4]:
binary_masks = [cv.imread(os.path.join(PATH_TO_BINARY_MASKS, binary_mask_name))[..., ::-1][:,:,0] 
                for binary_mask_name in binary_mask_names]
# floodfilled_masks = [cv.imread(os.path.join(PATH_TO_FLOODFILLED_MASKS, floodfilled_mask_name))[..., ::-1][:,:,0] 
#                 for floodfilled_mask_name in binary_mask_names]

# Lumen Corection
### a) Lumen or tromboza otherwise

In [ ]:
lument_tromboza_otherwise = [
    '10251_14_226.png',
    '10251_14_228.png',
    '10251_14_230.png',
    '10251_14_232.png',
    '10251_14_236.png',
    '10251_14_238.png',
    '10251_14_2909.png',
    '10251_14_2911.png',
    '10251_14_2913.png',
    '10251_14_2917.png',
    '10764_15_352.png',
    '10764_15_3038.png',
    '10772_15_3126.png', # strange
    '10772_15_3128.png', # strange
    '11191_14_686.png',
    '2917_15_690.png',
    '2917_15_692.png',
    '2917_15_3401.png',
    '2917_15_3403.png',
    '2917_15_3407.png',
    '2917_15_3409.png',
    '10772_15_436.png',
    #---- Cases, when need to correct the boundaries ---------
#     '10764_15_356.png',
#     '10764_15_3042.png',
#     '11191_14_672.png',
#     '11191_14_678.png',
#     '11191_14_3386.png',
#     '11191_14_3388.png',
#     '516_15_4330.png',
#     '516_15_4336.png',
#     '3315_15_2056.png',
#     '3315_15_2072.png',
#     '9447_14_1763.png',
#     '9447_14_4531.png',
#     '9447_14_4537.png'
]

In [ ]:
def concatenate_binary_and_original(binary_img, original):
    new_result = np.copy(original)
    new_result[new_result == 6] = 9
    kernel = np.ones((5,5))
    opened_binary_img = cv.morphologyEx(binary_img, cv.MORPH_OPEN, kernel)
    replaced_area = np.logical_and(new_result == 9,binary_img).astype('uint8')
    replaced_area[replaced_area==1] = 6
    new_result[replaced_area == 6] = 6
    return new_result

In [ ]:
PATH_TO_SAVE_MODIFIED_LUMEN = '/home/bohdan/histologie/data/doctor_annotations/masks_corrections/lumen_correction/'

for i in range(len(floodfilled_masks)):
    if binary_mask_names[i] in lument_tromboza_otherwise:
        Image.fromarray(concatenate_binary_and_original(binary_masks[i], floodfilled_masks[i])).save(os.path.join(PATH_TO_SAVE_MODIFIED_LUMEN, binary_mask_names[i]))


### b) Copy the rest of the images

In [ ]:
for i in range(len(binary_mask_names)):
    if binary_mask_names[i] not in lument_tromboza_otherwise:
        copyfile(
                os.path.join(COMMON_PART, 'colored_masks', binary_mask_names[i]), 
                os.path.join(COMMON_PART, 'masks_corrections/lumen_correction/kik',binary_mask_names[i])
        )

# Ateroma tkan Correction

In [ ]:
PATH_TO_LUMEN_EDITED = '/home/bohdan/histologie/data/doctor_annotations/masks_corrections/lumen_correction/unvisible_masks/'
lumen_edited_names = os.listdir(PATH_TO_LUMEN_EDITED)
lumen_edited_imgs = [cv.imread(os.path.join(PATH_TO_LUMEN_EDITED, binary_mask_name))[..., ::-1][:,:,0] 
                for binary_mask_name in binary_mask_names]

In [ ]:
def concatenate_binary_and_original_tromboza(binary_img, original, class_to_substitute):
    new_result = np.copy(original)
    kernel = np.ones((5,5))
    opened_binary_img = cv.morphologyEx(binary_img, cv.MORPH_OPEN, kernel)
    replaced_area = np.logical_and(new_result == class_to_substitute,binary_img).astype('uint8')
    replaced_area[replaced_area==1] = class_to_substitute
    new_result[new_result==class_to_substitute]=0
    new_result[replaced_area == class_to_substitute] = class_to_substitute
    return new_result

In [ ]:
edited_ateroma = [
    concatenate_binary_and_original_tromboza(binary_masks[i],lumen_edited_imgs[i],1) for 
    i in range(len(binary_mask_names))]

In [ ]:
PATH_TO_SAVE_ATEROMA = '/home/bohdan/histologie/data/doctor_annotations/masks_corrections/ateroma_corection/unvisible/'

for i in range(len(binary_mask_names)):
    Image.fromarray(edited_ateroma[i]).save(os.path.join(PATH_TO_SAVE_ATEROMA, binary_mask_names[i]))


# Fibro Correction

In [ ]:
PATH_TO_ATEROMA = '/home/bohdan/histologie/data/doctor_annotations/masks_corrections/ateroma_corection/unvisible/'
ateroma_edited_names = os.listdir(PATH_TO_ATEROMA)
ateroma_edited_imgs = [cv.imread(os.path.join(PATH_TO_ATEROMA, binary_mask_name))[..., ::-1][:,:,0] 
                for binary_mask_name in binary_mask_names]

In [ ]:
edited_fibro = [
    concatenate_binary_and_original_tromboza(binary_masks[i],ateroma_edited_imgs[i],2) for 
    i in range(len(binary_mask_names))]

In [ ]:
PATH_TO_SAVE_FIBRO = '/home/bohdan/histologie/data/doctor_annotations/masks_corrections/fibro_correction/unvisible/'

for i in range(len(binary_mask_names)):
    Image.fromarray(edited_fibro[i]).save(os.path.join(PATH_TO_SAVE_FIBRO, binary_mask_names[i]))


# Unknown correction

In [5]:
PATH_TO_FIBRO = '/home/bohdan/histologie/data/doctor_annotations/masks_corrections/fibro_correction/unvisible/'
fibro_edited_names = os.listdir(PATH_TO_FIBRO)
fibro_edited_imgs = [cv.imread(os.path.join(PATH_TO_FIBRO, binary_mask_name))[..., ::-1][:,:,0] 
                for binary_mask_name in binary_mask_names]

In [18]:
def add_unknown_to_tissue_areas(img, binary_img):
    new_result = np.copy(img)
    kernel = np.ones((5,5))
    opened_binary_img = cv.morphologyEx(np.copy(binary_img), cv.MORPH_OPEN, kernel)
    opened_binary_img[new_result != 0] = 0
    new_result[opened_binary_img!=0] = 13
    return new_result

In [21]:
edited_unknown = [add_unknown_to_tissue_areas(fibro_edited_imgs[i], binary_masks[i]) 
    for i in range(len(binary_mask_names))]

In [26]:
PATH_TO_SAVE_UNKNOWN = '/home/bohdan/histologie/data/doctor_annotations/masks_corrections/unknown_correction/unvisible/'

for i in range(len(binary_mask_names)):
    Image.fromarray(edited_unknown[i]).save(os.path.join(PATH_TO_SAVE_UNKNOWN, binary_mask_names[i]))


# Make Unknown the rest of classes


In [6]:
PATH_TO_UNKNOWN = '/home/bohdan/histologie/data/doctor_annotations/masks_corrections/unknown_correction/unvisible/'
unknown_edited_names = os.listdir(PATH_TO_UNKNOWN)
unknown_edited_imgs = [cv.imread(os.path.join(PATH_TO_UNKNOWN, binary_mask_name))[..., ::-1][:,:,0] 
                for binary_mask_name in binary_mask_names]

In [58]:
def reset_rest_classes_to_unknown(img):
    result = np.copy(img)
    classes_to_unkwnown = np.isin(result,[3,4,5,7,8,10,11,12])
    result[classes_to_unkwnown] = 13
    return result

In [65]:
edited_rest_classes = [reset_rest_classes_to_unknown(img) for img in unknown_edited_imgs]

In [66]:
PATH_TO_SAVE_REST = '/home/bohdan/histologie/data/doctor_annotations/masks_corrections/final_corrections_5/unvisible'

for i in range(len(binary_mask_names)):
    Image.fromarray(edited_rest_classes[i]).save(os.path.join(PATH_TO_SAVE_REST, binary_mask_names[i]))
